In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt
from IPython.display import clear_output
import tensorflow_hub as hub

In [ ]:
train_data_dir = '/kaggle/input/soybean-diseased-leaf-dataset'
BATCH_SIZE = 32
IMG_SIZE = (224, 224)
EPOCHS = 25
SHUFFLE_BUFFER = 1000
#PRETRAINED_MODEL_PATH = "https://www.kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-classification/versions/2"
validation_split = 0.2 

In [ ]:
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    seed=42,
    validation_split=validation_split, 
    subset='training',  
)

validation_dataset = tf.keras.utils.image_dataset_from_directory(
    train_data_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    labels='inferred',
    label_mode='categorical',
    shuffle=True,
    seed=42,
    validation_split=validation_split,  
    subset='validation',  
)

In [ ]:
class_names = train_dataset.class_names


plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
    for i in range(9):
        image = images[i].numpy()  
        label = labels[i]
        class_name = class_names[tf.argmax(label)]
        plt.subplot(3, 3, i + 1)
        plt.imshow(image.astype("uint8"))  
        plt.title(class_name)
        plt.axis("off")  
plt.show()

## Data augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.Rescaling(1/255.)
])
normalization = layers.Rescaling(1/255.)

In [ ]:
training_data = (train_dataset.map(lambda x,y: (data_augmentation(x), y))
                 .shuffle(SHUFFLE_BUFFER).prefetch(1).cache())
validation_data = (validation_dataset.map(lambda x,y: (normalization(x), y))
                   .prefetch(1).cache())   

# Loading Inception-V3 model

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import plot_model

In [ ]:
#use all except for the last layer (specific to the 'imagenet' competition)
pretrained_model = InceptionV3(input_shape = (224, 224,3), 
                              include_top = False,
                              weights = 'imagenet')

In [ ]:
#plot_model(pretrained_model, show_shapes=True)

In [ ]:
#make all layer non-trainable 
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
rmsprop = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath='weights.ckpt',
    save_best_only=True,
    save_weights_only=True,
    monitor='val_loss')
]

In [ ]:
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = pretrained_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(224, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(len(class_names), activation='softmax')(x)

model = keras.Model(inputs, x)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
             optimizer=rmsprop,
             metrics=['accuracy'])

history = model.fit(training_data, 
                    epochs=EPOCHS,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
results = model.evaluate(validation_data)

print("Validation Loss:", results[0])
print("Validation Accuracy:", results[1])

In [ ]:
num_rows = 5
num_cols = 4
plt.figure(figsize=(12, 12))

for images, labels in validation_dataset:
    images_normalized = normalization(images)
    
    predictions = model.predict(images_normalized)
    
    for i in range(images.shape[0]):
        image = images[i].numpy()
        label = labels[i]
        

        predicted_class_index = tf.argmax(predictions[i])
        predicted_class_name = class_names[predicted_class_index]
        
        plt.subplot(num_rows, num_cols, i + 1)
        plt.imshow(image.astype("uint8"))
        plt.title(f"Predicted: {predicted_class_name}\nTrue: {class_names[tf.argmax(label)]}")
        plt.axis("off")
        
        if i == num_rows * num_cols - 1:
            break

    plt.tight_layout()
    plt.show()
    
    if i == num_rows * num_cols - 1:
        break

